In [3]:

import spotipy
from spotipy.oauth2 import SpotifyOAuth


client_id = 'ADD CLIENT ID'
client_secret = 'ADD CLIENT SECRET'
redirect_uri = 'ADD REDIRECT URL'

scope = 'playlist-read-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))

def scraper(name):
    target_playlist_name = name
    playlist_id = None

    # Get current playlist
    playlists = sp.current_user_playlists()

    for playlist in playlists['items']:
        if playlist['name'] == target_playlist_name:
            playlist_id = playlist['id']
            break

    if not playlist_id:
        print(f"Playlist '{target_playlist_name}' not found.")
        return []

    print(f"Scraping playlist: {target_playlist_name}")

    offset = 0
    limit = 100  # Maximum possible per request

    S_playlist = []  # List to store song names and artist names to use in the youtube side

    while True:
        results = sp.playlist_tracks(playlist_id, offset=offset, limit=limit)
        for idx, item in enumerate(results['items']):
            track = item['track']
            track_name = f"{idx + 1}. {track['name']} - {track['artists'][0]['name']}"
            S_playlist.append(f"{track['name']} - {track['artists'][0]['name']}")
            print(track_name)  # Print track name

        offset += limit
        # Check if there are more items to fetch
        if not results['next']:
            break  # Exit loop if no more results

    return S_playlist

Scraping playlist: B
1. double take - Dhruv
2. Lucid Dreams - Juice WRLD
3. Lucid Dreams - Juice WRLD
4. SAD! - XXXTENTACION
5. All Girls Are The Same - Juice WRLD
6. NUMB - XXXTENTACION
7. Moonlight - XXXTENTACION
8. Without Me - Halsey
9. goosebumps - Travis Scott
10. Hope - XXXTENTACION
11. Falling - Trevor Daniel
12. Robbery - Juice WRLD
13. idfc - Tarro Remix - blackbear
14. See Me Fall - Y2K Remix - Ro Ransom
15. Hate Me - Ellie Goulding
16. Shinunoga E-Wa - Fujii Kaze
17. Agoraphobic Nights - I Am Hand
18. Smile (with The Weeknd) - Juice WRLD
19. Wishing Well - Juice WRLD
20. Empty - Juice WRLD
21. Empty Space - James Arthur
22. Falling (feat. blackbear) - blackbear Remix - Trevor Daniel
23. idfc - blackbear
24. IDGAF (with blackbear) - BoyWithUke
25. idfc - Tarro Remix - blackbear
26. veil - 須田景凪
27. Dive Back In Time - 动画《时光代理人》片头曲 - BaishaJAWS
28. 0 (zero) - LMYK
29. Akuma no Ko - Higuchi Ai
30. カタオモイ - From THE FIRST TAKE - Aimer
31. おもかげ (produced by Vaundy) - milet
32. 残響散

In [5]:
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.firefox.webdriver import WebDriver as FirefoxWebDriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time

def init_driver():
    firefox_profile_path = 'ADD YOUR PROFILE PATH GOOGLE ACCOUNT LOGGED IN'
    service = FirefoxService(executable_path=GeckoDriverManager().install())
    options = webdriver.FirefoxOptions()
    options.add_argument('-profile')
    options.add_argument(firefox_profile_path)
    driver = FirefoxWebDriver(service=service, options=options)
    return driver

def create_playlist(driver, playlist_name):
    description = "This playlist exists by the grace and wisdom of Rohit's Python Script"
    try:
        # Click the "+ New Playlist" button
        new_playlist_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="New playlist"]'))
        )
        new_playlist_button.click()

        # Wait for the playlist form to appear
        time.sleep(2)

        # Fill in the title
        title_input = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//tp-yt-paper-input[@id="title-input"]//input'))
        )
        title_input.click()
        time.sleep(1)
        title_input.send_keys(playlist_name)

        # Fill in the description
        description_input = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//tp-yt-paper-input-container[@class="style-scope tp-yt-paper-textarea"]//textarea'))
        )
        description_input.click()
        description_input.send_keys(description)

        create_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Create"]'))
        )
        create_button.click()

        print(f"Created playlist '{playlist_name}' with description '{description}'")
        time.sleep(2)

    except Exception as e:
        print(f"Error creating playlist '{playlist_name}': {e}")

In [20]:
import time



def add_to_playlist(driver, playlist_name, playlist, start_index=0):
    not_found_songs = []
    
    try:
        # Wait until the search box is present and interactable
        search_box = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "ytmusic-search-box input")))
        
        # List of words to search
        search_words = playlist
        
        
        for word in search_words[start_index:]:
            st=time.time()

            
            try:
                
                # Click the search box to ensure it's focused
                search_box.click()
            
                # Clear any previous input in the search box
                search_box.clear()
            
                # Enter the search word
                search_box.send_keys(word)
                
                # Submit the form
                search_box.send_keys(Keys.RETURN)
                
                #Why this new_playlist_wait?
                #The components in the next section exist for all web searches so if the code doesn't wait until the next page is loaded it'll try to save a video it already saved 
                # since right after saving it will look for the save button and if not found right click the first video and save it all of this might be faster than the next page can be loaded
                new_playlist_button = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//button[@aria-label="New playlist"]')))
               
                
                try:
                    # If the save button is found we just click and add
                    save_button = WebDriverWait(driver, 1).until(
                        EC.presence_of_element_located((By.XPATH, '//div[@class="yt-spec-button-shape-next__button-text-content"]//span[text()="Save"]'))
                    )
        
                    # Scroll into view and click using JavaScript
                    driver.execute_script("arguments[0].scrollIntoView(true);", save_button)
                    driver.execute_script("arguments[0].click();", save_button)
                    print("Save button found")
                except:
                    # If the save button isn't found we find the first video and right click that and then click save to playlist
                    print("Save button not found going with first video option")
                    first_video = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.XPATH, '/html/body/ytmusic-app/ytmusic-app-layout/div[4]/ytmusic-search-page/ytmusic-tabbed-search-results-renderer/div[2]/ytmusic-section-list-renderer/div[2]/ytmusic-shelf-renderer[1]/div[3]/ytmusic-responsive-list-item-renderer[1]'))
                    )

                    # Right-click on the first video
                    actions = ActionChains(driver)
                    actions.context_click(first_video).perform()

                    # Wait until the "Save to playlist" option is visible and click it
                    save_to_playlist_option = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.XPATH, '/html/body/ytmusic-app/ytmusic-popup-container/tp-yt-iron-dropdown/div/ytmusic-menu-popup-renderer/tp-yt-paper-listbox/ytmusic-menu-navigation-item-renderer[1]'))
                    )
                    save_to_playlist_option.click()
                
                try:
                    # Locate the playlist by title attribute and click the corresponding button
                    playlist_button = WebDriverWait(driver, 5).until(
                       EC.element_to_be_clickable((By.XPATH, f'//div[@id="playlists"]//ytmusic-playlist-add-to-option-renderer//button[@aria-label="{playlist_name} "]'))
                    )
                    playlist_button.click()
                    time.sleep(3)
                    
                    
                    print(f"Added first result of '{word}' to playlist '{playlist_name}'")
                except Exception as e:
                    print(f"Playlist not shown for '{word}', moving to next song")
                    
            
                
                
            except Exception as e:
                print(f"Error processing '{word}': {e}")
                not_found_songs.append(word)
                
                continue
            et=time.time()
            
            print(f"Time taken {et-st}")
            
            
        
    except Exception as e:
        print(f"Error adding to playlist '{playlist_name}': {e}")
        # Optionally, print page source for debugging
        print(f"Page source: {driver.page_source}")
        
    
    return not_found_songs

def S_to_Y_Playlister(Spotify_list, Youtube_playlist,start_index=0):
    driver = init_driver()
    try:
        # Open YouTube Music
        driver.get('https://music.youtube.com')

    
        time.sleep(3) 

        # Create the YouTube playlist
        create_playlist(driver, Youtube_playlist)

        # Add songs to the created playlist
        songs_not_found=add_to_playlist(driver, Youtube_playlist, Spotify_list,start_index=start_index)

    finally:
        driver.quit()
        return songs_not_found
    







36